In [1]:
%env CUDA_VISIBLE_DEVICES=1

env: CUDA_VISIBLE_DEVICES=1


In [2]:
from huggingface_hub import notebook_login

notebook_login()

In [19]:
from datasets import load_dataset, load_metric, Audio

common_voice_train = load_dataset("mozilla-foundation/common_voice_13_0", "bn", split="train", use_auth_token=True)
common_voice_test = load_dataset("mozilla-foundation/common_voice_13_0", "bn", split="test", use_auth_token=True)

Found cached dataset common_voice_13_0 (/home/dev/.cache/huggingface/datasets/mozilla-foundation___common_voice_13_0/bn/13.0.0/2506e9a8950f5807ceae08c2920e814222909fd7f477b74f5d225802e9f04055)
Found cached dataset common_voice_13_0 (/home/dev/.cache/huggingface/datasets/mozilla-foundation___common_voice_13_0/bn/13.0.0/2506e9a8950f5807ceae08c2920e814222909fd7f477b74f5d225802e9f04055)


In [20]:
common_voice_train = common_voice_train.remove_columns(["accent", "age", "client_id", "down_votes", "gender", "locale", "segment", "up_votes", "variant"])
common_voice_test = common_voice_test.remove_columns(["accent", "age", "client_id", "down_votes", "gender", "locale", "segment", "up_votes", "variant"])

In [21]:
import re
chars_to_remove_regex = '[\,\?\.\!\-\;\:\"\“\%\‘\”\�\'\‚\/]'

def remove_special_characters(batch):
    batch["sentence"] = re.sub(chars_to_remove_regex, '', batch["sentence"]).lower()
    return batch

In [22]:
common_voice_train = common_voice_train.map(remove_special_characters)
common_voice_test = common_voice_test.map(remove_special_characters)

Map:   0%|          | 0/20729 [00:00<?, ? examples/s]

Map:   0%|          | 0/9230 [00:00<?, ? examples/s]

In [24]:
import pandas as pd
from datasets import DatasetDict
labels_df = pd.read_csv("bengaliai-speech/train.csv")

In [25]:
from sklearn.model_selection import train_test_split
train_df = labels_df[labels_df["split"]=="train"]
val_df = labels_df[labels_df["split"]=="valid"]

In [26]:
import pandas as pd
import os
from torch.utils.data import Dataset
from datasets import Audio

class AudioDataset(Dataset):
    def __init__(self, labels_df: pd.DataFrame, data_path: str, sampling_rate: int =32_000):
        self.labels_df = labels_df
        self.data_path = data_path
        self.sampling_rate = sampling_rate
    def __len__(self):
        return len(self.labels_df)
    def __getitem__(self,idx):
        row = self.labels_df.iloc[idx]
        path = os.path.join(self.data_path, row["id"]+".mp3")
        sentence = row["sentence"]
        with open(path, "rb") as f:
            speech = f.read()
            audio = Audio(sampling_rate=self.sampling_rate).decode_example({"path": path, "bytes": speech})

        return {"audio": audio, "sentence": sentence}
        # example = processor(audio=audio["array"], sampling_rate=processor.feature_extractor.sampling_rate, text=sentence, return_tensors="pt")
        # example["input_values"] = example["input_values"][0]
        # example["labels"] = example["labels"][0]
        # example["input_length"] = len(audio["array"]) // processor.feature_extractor.sampling_rate
        # speech, sr = librosa.load(path, sr=processor.feature_extractor.sampling_rate) 
#         print(speech.shape)
        # return example

In [27]:
train_dataset= AudioDataset(train_df, data_path="bengaliai-speech/train_mp3s" )
test_dataset = AudioDataset(val_df, data_path="bengaliai-speech/train_mp3s" )

In [31]:
from datasets import Dataset
from functools import partial

def gen(dset):
    for idx in range(len(dset)):
        yield dset[idx]  # this has to be a dictionary
    ## or if it's an IterableDataset
    # for ex in torch_dataset:
    #     yield ex

train_gen = partial(gen, train_dataset)
test_gen = partial(gen, test_dataset)

train_dset = Dataset.from_generator(train_gen)

Generating train split: 0 examples [00:00, ? examples/s]

Dataset generator downloaded and prepared to /home/dev/.cache/huggingface/datasets/generator/default-c81e77ba63ca32e0/0.0.0. Subsequent calls will reuse this data.


In [32]:
test_dset = Dataset.from_generator(test_gen)

Generating train split: 0 examples [00:00, ? examples/s]

Dataset generator downloaded and prepared to /home/dev/.cache/huggingface/datasets/generator/default-e29556b32b5756f1/0.0.0. Subsequent calls will reuse this data.


In [ ]:
huggingface_dset = DatasetDict({"train":train_dset,"validation": test_dset})

In [34]:

huggingface_dset.push_to_hub("Kurokabe/bengaliai-speech", private=True)

Pushing split train to the Hub.
Resuming upload of the dataset shards.


Pushing dataset shards to the dataset hub:   0%|          | 0/2086 [00:00<?, ?it/s]

Deleting unused files from dataset repository:   0%|          | 0/818 [00:00<?, ?it/s]

HfHubHTTPError: 429 Client Error: Too Many Requests for url: https://huggingface.co/api/datasets/Kurokabe/bengaliai-speech/commit/main (Request ID: Root=1-64ccab0d-52308e763136ea3d5cc13f1e;ab145d5d-30f5-4668-b307-7d11ba4e0f7f)

You have exceeded our hourly quotas for action: commit. We invite you to retry later.

In [42]:
Dataset.from_dict(huggingface_dset.cache_files["train"])

AttributeError: 'list' object has no attribute 'items'

In [45]:
huggingface_dset.save_to_disk("bengaliai-speech_huggingface")

Saving the dataset (0/2086 shards):   0%|          | 0/934048 [00:00<?, ? examples/s]

Saving the dataset (0/91 shards):   0%|          | 0/29588 [00:00<?, ? examples/s]

In [47]:
DatasetDict.load_from_disk("bengaliai-speech_huggingface")

DatasetDict({
    train: Dataset({
        features: ['audio', 'sentence'],
        num_rows: 934048
    })
    validation: Dataset({
        features: ['audio', 'sentence'],
        num_rows: 29588
    })
})